In [314]:
# pip install eralchemy

import sqlite3
import pandas as pd
from IPython.display import display


In [315]:
database = 'data/mlb_data.sqlite'
conn = sqlite3.connect(database)
# cursor = conn.cursor()

In [316]:
query = 'SELECT player_id, ' \
        'SUM(g_c) as catcher, ' \
        'SUM(g_1b) as firstbaseman, ' \
        'SUM(g_2b) as secondbaseman, ' \
        'SUM(g_3b) as thirdbaseman, ' \
        'SUM(g_ss) as shortstop, ' \
        'SUM(g_of) as outfielder ' \
        'from appearances group by player_id'
plyr_pos_df = pd.read_sql_query(query, conn)
plyr_pos_df.head()

,player_id,catcher,firstbaseman,secondbaseman,thirdbaseman,shortstop,outfielder
0,aardsda01,0,0,0,0,0,0
1,aaronha01,0,210,43,7,0,2760
2,aaronto01,0,232,7,10,0,138
3,aasedo01,0,0,0,0,0,0
4,abadan01,0,8,0,0,0,1


In [317]:
# Number of gams in 1 MLB season today (and since 1961) is 162, it was 140 in 1901 and 154 in 1920
positions = ['catcher', 'firstbaseman', 'secondbaseman', 'thirdbaseman', 'shortstop', 'outfielder']
min_games_at_max_pos = plyr_pos_df[positions].max(axis=1) > 140
plyr_pos_df = plyr_pos_df[min_games_at_max_pos]

In [318]:
plyr_pos_df['position'] = plyr_pos_df[positions].idxmax(axis=1)
plyr_pos_df.set_index('player_id', inplace=True)
plyr_pos_df.head()

,catcher,firstbaseman,secondbaseman,thirdbaseman,shortstop,outfielder,position
player_id,,,,,,,
aaronha01,0,210,43,7,0,2760,outfielder
aaronto01,0,232,7,10,0,138,firstbaseman
abbated01,0,0,419,20,388,3,secondbaseman
abbeych01,0,0,0,0,0,451,outfielder
abbotje01,0,0,0,0,0,185,outfielder


In [319]:
player_list = plyr_pos_df.index.tolist()

In [320]:
query = "SELECT player_id, name_first ||' ' || name_last AS name, debut, weight, height, bats as bat_hand, throws as throw_hand FROM player"
players_df = pd.read_sql_query(query, conn)
player_mask = players_df['player_id'].isin(player_list)
players_df = players_df[player_mask]
players_df.set_index('player_id', inplace=True)
display(players_df.head())
# players_df.info()

,name,debut,weight,height,bat_hand,throw_hand
player_id,,,,,,
aaronha01,Hank Aaron,1954-04-13,180,72,R,R
aaronto01,Tommie Aaron,1962-04-10,190,75,R,R
abbated01,Ed Abbaticchio,1897-09-04,170,71,R,R
abbeych01,Charlie Abbey,1893-08-16,169,68,L,L
abbotje01,Jeff Abbott,1997-06-10,190,74,R,L


In [321]:
players_df = pd.concat([players_df, plyr_pos_df['position']], axis=1)
players_df.head()

,name,debut,weight,height,bat_hand,throw_hand,position
player_id,,,,,,,
aaronha01,Hank Aaron,1954-04-13,180,72,R,R,outfielder
aaronto01,Tommie Aaron,1962-04-10,190,75,R,R,firstbaseman
abbated01,Ed Abbaticchio,1897-09-04,170,71,R,R,secondbaseman
abbeych01,Charlie Abbey,1893-08-16,169,68,L,L,outfielder
abbotje01,Jeff Abbott,1997-06-10,190,74,R,L,outfielder


In [322]:
print(players_df.isnull().sum())
print(players_df.info())

name          0
debut         0
weight        0
height        0
bat_hand      0
throw_hand    0
position      0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 4371 entries, aaronha01 to zwilldu01
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        4371 non-null   object
 1   debut       4371 non-null   object
 2   weight      4371 non-null   object
 3   height      4371 non-null   object
 4   bat_hand    4371 non-null   object
 5   throw_hand  4371 non-null   object
 6   position    4371 non-null   object
dtypes: object(7)
memory usage: 273.2+ KB
None


In [323]:
missing_height_or_weight = (players_df['height'] == '') | (players_df['weight'] == '')
display(players_df[missing_height_or_weight])

missing_bat_or_throw_hand = (players_df['bat_hand'] == '') | (players_df['throw_hand'] == '')
display(players_df[missing_bat_or_throw_hand])

print(players_df['bat_hand'].value_counts())
print(players_df['throw_hand'].value_counts())

,name,debut,weight,height,bat_hand,throw_hand,position
player_id,,,,,,,
barrysh01,Shad Barry,1899-05-30,,,R,R,outfielder
birchju01,Jud Birchall,1882-05-02,,,,,outfielder
boothed01,Eddie Booth,1872-04-26,,,,,outfielder
creamge01,George Creamer,1878-05-01,,74,R,R,secondbaseman
dehlmhe01,Herman Dehlman,1872-05-02,,,,,firstbaseman
dunnja01,Jack Dunn,1897-05-06,,69,R,R,thirdbaseman
edench01,Charlie Eden,1877-08-17,168,,L,L,outfielder
fostepo01,Pop Foster,1898-09-13,,68,R,R,outfielder
gleasja01,Jack Gleason,1877-10-02,170,,R,R,thirdbaseman


,name,debut,weight,height,bat_hand,throw_hand,position
player_id,,,,,,,
bechtge01,George Bechtel,1871-05-20,165,71,,,outfielder
bergema01,Marty Bergen,1896-04-17,170,70,,R,catcher
birchju01,Jud Birchall,1882-05-02,,,,,outfielder
boothed01,Eddie Booth,1872-04-26,,,,,outfielder
bradyst01,Steve Brady,1874-07-23,165,69,,,outfielder
briodfa01,Fatty Briody,1880-06-16,190,68,,R,catcher
burnsji01,Jim Burns,1888-09-25,168,67,R,,outfielder
childpe01,Pete Childs,1901-04-24,175,67,,R,secondbaseman
dehlmhe01,Herman Dehlman,1872-05-02,,,,,firstbaseman


R    2560
L    1353
B     418
       40
Name: bat_hand, dtype: int64
R    3744
L     584
       43
Name: throw_hand, dtype: int64


In [324]:
players_df = players_df[~missing_height_or_weight & ~missing_bat_or_throw_hand]
players_df['debut'] = pd.to_datetime(players_df['debut'])
players_df['rookie_year'] = players_df['debut'].dt.year
players_df.drop(['debut'] , axis=1, inplace=True)
players_df['height'] = players_df['height'].astype(int)
players_df['weight'] = players_df['weight'].astype(int)


In [325]:
print(players_df.info())
print(players_df.shape)
display(players_df.head(10))

<class 'pandas.core.frame.DataFrame'>
Index: 4290 entries, aaronha01 to zwilldu01
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         4290 non-null   object
 1   weight       4290 non-null   int64 
 2   height       4290 non-null   int64 
 3   bat_hand     4290 non-null   object
 4   throw_hand   4290 non-null   object
 5   position     4290 non-null   object
 6   rookie_year  4290 non-null   int64 
dtypes: int64(3), object(4)
memory usage: 268.1+ KB
None
(4290, 7)


,name,weight,height,bat_hand,throw_hand,position,rookie_year
player_id,,,,,,,
aaronha01,Hank Aaron,180,72,R,R,outfielder,1954
aaronto01,Tommie Aaron,190,75,R,R,firstbaseman,1962
abbated01,Ed Abbaticchio,170,71,R,R,secondbaseman,1897
abbeych01,Charlie Abbey,169,68,L,L,outfielder,1893
abbotje01,Jeff Abbott,190,74,R,L,outfielder,1997
abbotku01,Kurt Abbott,180,71,R,R,shortstop,1993
abernbr01,Brent Abernathy,185,73,R,R,secondbaseman,2001
abnersh01,Shawn Abner,190,73,R,R,outfielder,1987
abramca01,Cal Abrams,185,72,L,L,outfielder,1949


In [326]:
# players_df = pd.get_dummies(players_df, columns=['position','bat_hand', 'throw_hand'])
# display(players_df.head(10))

In [327]:
query = "SELECT player_id, year, stint, g as games, ab as at_bats, h as hits, hr, rbi FROM batting"
performance_df = pd.read_sql_query(query, conn)
player_mask = performance_df['player_id'].isin(player_list)
performance_df = performance_df[player_mask]
performance_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45568 entries, 1 to 101330
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   player_id  45568 non-null  object
 1   year       45568 non-null  int64 
 2   stint      45568 non-null  int64 
 3   games      45568 non-null  int64 
 4   at_bats    45568 non-null  object
 5   hits       45568 non-null  object
 6   hr         45568 non-null  object
 7   rbi        45568 non-null  object
dtypes: int64(3), object(5)
memory usage: 3.1+ MB


In [328]:
performance_df['at_bats'] = performance_df['at_bats'].astype(int)
necessary_at_bats = performance_df['at_bats'] > 502
performance_df = performance_df[necessary_at_bats]

performance_df['hits'] = performance_df['hits'].astype(int)
performance_df['hr'] = performance_df['hr'].astype(int)
performance_df['rbi'] = performance_df['rbi'].astype(int)
performance_df['ba'] = performance_df['hits'] / performance_df['at_bats']

display(performance_df.shape)
display(performance_df['stint'].value_counts())
display(performance_df.groupby(['player_id', 'year']).ngroups)
## No Player-Year combo has more than one stint with > 502 at bats; hence we can drop stint
performance_df.drop(['stint', 'at_bats', 'hits'] , axis=1, inplace=True)
performance_df.info()

(8468, 9)

1    8435
2      33
Name: stint, dtype: int64

8468

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8468 entries, 2141 to 101305
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   player_id  8468 non-null   object 
 1   year       8468 non-null   int64  
 2   games      8468 non-null   int64  
 3   hr         8468 non-null   int64  
 4   rbi        8468 non-null   int64  
 5   ba         8468 non-null   float64
dtypes: float64(1), int64(4), object(1)
memory usage: 463.1+ KB


In [330]:
performance_df.set_index(['player_id', 'year'], inplace=True)
performance_df.sort_index(inplace=True)
display(performance_df.head(75))

games  hr  rbi        ba
player_id year                          
aaronha01 1955    153  27  106  0.313953
          1956    153  26   92  0.328407
          1957    151  44  132  0.321951
          1958    153  30   95  0.326123
          1959    154  39  123  0.354531
...               ...  ..  ...       ...
allendi01 1964    162  29   91  0.318038
          1965    161  20   85  0.302100
          1966    141  40  110  0.316794
          1968    152  33   90  0.262956
          1971    155  23   90  0.295082

[75 rows x 4 columns]

In [334]:
performance_df['cum_games'] = performance_df.groupby(level='player_id')['games'].cumsum()

In [335]:
display(performance_df.head(75))

games  hr  rbi        ba  cum_games
player_id year                                     
aaronha01 1955    153  27  106  0.313953        153
          1956    153  26   92  0.328407        306
          1957    151  44  132  0.321951        457
          1958    153  30   95  0.326123        610
          1959    154  39  123  0.354531        764
...               ...  ..  ...       ...        ...
allendi01 1964    162  29   91  0.318038        162
          1965    161  20   85  0.302100        323
          1966    141  40  110  0.316794        464
          1968    152  33   90  0.262956        616
          1971    155  23   90  0.295082        771

[75 rows x 5 columns]